In [70]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

In [71]:
#PCSI0044

In [72]:
#Read the data

GenomicFeatures_PCSI0044 = pd.read_csv(open('data/PCSI0044/GenomicFeatures_PCSI0044.txt','r'),sep='\t')
SNVCalls_PCSI0044    = pd.read_csv(open('data/PCSI0044/SNVCalls_PCSI0044.txt','r'),sep='\t') 

In [73]:
print GenomicFeatures_PCSI0044.shape
print SNVCalls_PCSI0044.shape

(118394, 15)
(118394, 129)


In [74]:
np.sum(GenomicFeatures_PCSI0044.isnull())

CHROM                  0
POS                    0
RefAllele              0
NonRefAllele           0
BaseQual               0
TumourCoverage         0
NormalCoverage         0
MapQual                0
ReadPosition          77
Trinucleotide          0
HomopolymerRate        0
GCcontent              0
GenomicLocation    39909
DistanceSNP            0
StrandBias            77
dtype: int64

In [75]:
GenomicFeatures_PCSI0044['ReadPosition'].fillna(0, inplace=True)  #Replace na with 0
GenomicFeatures_PCSI0044['StrandBias'].fillna(0, inplace=True)

#print np.sum(GenomicFeatures_PCSI0023.isnull())

In [76]:
np.unique(GenomicFeatures_PCSI0044['GenomicLocation'])

array(['UTR3', 'UTR5', 'downstream', 'exonic', 'exonic;splicing',
       'intergenic', 'intronic', 'nan', 'ncRNA_UTR3', 'ncRNA_UTR5',
       'ncRNA_exonic', 'ncRNA_intronic', 'ncRNA_splicing', 'splicing',
       'upstream', 'upstream;downstream'], 
      dtype='|S19')

In [77]:
GenomicFeatures_PCSI0044.head()

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10038,522,12,16153,620,961,17,22,TTA,62.057971,0.343284,NaN,20885,0.820968
1,chr1,10109,362,152,9437,614,871,17,81,TTG,4.730000,0.507463,NaN,20814,0.749186
2,chr1,10177,122,54,3685,185,219,17,45,GTT,4.838384,0.502488,NaN,20746,0.281081
3,chr1,10180,146,33,4054,193,233,17,46,CTA,4.897959,0.497512,NaN,20743,0.264249
4,chr1,15190,185,97,8002,287,172,17,35,CCG,2.750000,0.641791,ncRNA_intronic,15733,0.463415


In [78]:
SNVCalls_PCSI0044.head()

,CHROM,POS,END,X2399008,X2701035,X2701039,X2762073,X2762075,X2791030,X2811081,X2811083,X2811087,X2811089,X2811393,X2812910,X2813249,X2813259,X2813291,X2813337,X2813381,
0,chr1,10037,10038,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,chr1,10108,10109,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...
2,chr1,10176,10177,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...
3,chr1,10179,10180,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...
4,chr1,15189,15190,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...


In [79]:
np.unique(GenomicFeatures_PCSI0044['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA', 'GCC',
       'GCG', 'GCT', 'GNC', 'GTA', 'GTC', 'GTG', 'GTT', 'TCA', 'TCC',
       'TCG', 'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [80]:
PCSI0044 = pd.merge(GenomicFeatures_PCSI0044, SNVCalls_PCSI0044,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [81]:
PCSI0044.head() #182+15

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399008,X2701035,X2701039,
0,chr1,10038,522,12,16153,620,961,17,22,TTA,62.057971,0.343284,NaN,20885,0.820968,10037,10038,0,1,1,...
1,chr1,10109,362,152,9437,614,871,17,81,TTG,4.730000,0.507463,NaN,20814,0.749186,10108,10109,0,0,0,...
2,chr1,10177,122,54,3685,185,219,17,45,GTT,4.838384,0.502488,NaN,20746,0.281081,10176,10177,0,0,0,...
3,chr1,10180,146,33,4054,193,233,17,46,CTA,4.897959,0.497512,NaN,20743,0.264249,10179,10180,0,0,0,...
4,chr1,15190,185,97,8002,287,172,17,35,CCG,2.750000,0.641791,ncRNA_intronic,15733,0.463415,15189,15190,0,0,0,...


In [82]:
PCSI0044_X = PCSI0044.iloc[:,2:15] #Exclude CHROM, POS
PCSI0044_Y = PCSI0044.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [83]:
print PCSI0044_X.shape
print PCSI0044_Y.shape

(118394, 13)
(118394, 126)


In [84]:
PCSI0044_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,522,12,16153,620,961,17,22,TTA,62.057971,0.343284,NaN,20885,0.820968
1,362,152,9437,614,871,17,81,TTG,4.730000,0.507463,NaN,20814,0.749186


In [85]:
PCSI0044_Y[:2]

,X2399008,X2701035,X2701039,X2762073,X2762075,X2791030,X2811081,X2811083,X2811087,X2811089,X2811393,X2812910,X2813249,X2813259,X2813291,X2813337,X2813381,X2813446,X2813711,X2813714,
0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...


In [86]:
totalSample = PCSI0044_Y.shape[0]
PCSI0044_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print PCSI0044_Y_Val.shape

118394
(118394, 1)


In [87]:
PCSI0044_Y.shape

(118394, 126)

In [88]:
PCSI0044_Y.values[0].shape

(126,)

In [89]:
for i in range(totalSample):
    PCSI0044_Y_Val[i] = np.argmax(np.bincount(PCSI0044_Y.values[i]))

In [90]:
PCSI0044_Y_Val_1 = pd.DataFrame(PCSI0044_Y_Val)
PCSI0044_Y_Val_1.columns = ['Label']

In [91]:
PCSI0044_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [92]:
PCSI0044_dataframe = pd.concat([PCSI0044_X, PCSI0044_Y_Val_1],axis=1)

In [93]:
PCSI0044_dataframe.shape

(118394, 14)

In [94]:
PCSI0044_dataframe[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,Label
0,522,12,16153,620,961,17,22,TTA,62.057971,0.343284,NaN,20885,0.820968,0
1,362,152,9437,614,871,17,81,TTG,4.730000,0.507463,NaN,20814,0.749186,0


In [39]:
PCSI0044_dataframe.to_csv('PCSI0044.csv')

In [40]:
#Now we have variable importance ...lets do analysis

In [41]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [43]:
%%R
# Read in the data
PCSI0044Frame = read.csv("PCSI0044.csv")
str(PCSI0044Frame)

'data.frame':	118394 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  522 362 122 146 185 130 148 120 253 90 ...
 $ NonRefAllele   : int  12 152 54 33 97 22 32 61 97 22 ...
 $ BaseQual       : int  16153 9437 3685 4054 8002 4592 5697 5812 9971 3126 ...
 $ TumourCoverage : int  620 614 185 193 287 157 186 187 357 113 ...
 $ NormalCoverage : int  961 871 219 233 172 118 90 90 210 101 ...
 $ MapQual        : num  17 17 17 17 17 16 16 16 37 0 ...
 $ ReadPosition   : num  22 81 45 46 35 67 57 59 67 59 ...
 $ Trinucleotide  : Factor w/ 33 levels "ACA","ACC","ACG",..: 30 32 25 13 11 10 9 31 18 19 ...
 $ HomopolymerRate: num  62.06 4.73 4.84 4.9 2.75 ...
 $ GCcontent      : num  0.343 0.507 0.502 0.498 0.642 ...
 $ GenomicLocation: Factor w/ 16 levels "","downstream",..: 1 1 1 1 8 1 8 8 1 1 ...
 $ DistanceSNP    : int  20885 20814 20746 20743 15733 14666 14428 14426 13385 11601 ...
 $ StrandBias     : num  0.821 0.749 0.281 0.264 0.463 ...
 $ Lab

In [44]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
#sum(is.na(IS1Frame$ReadPosition)) 
sum(is.na(PCSI0044Frame)) 

[1] 0


In [45]:
#%%R
#CPCG0235Frame$ReadPosition[is.na(CPCG0235Frame$ReadPosition)] = mean(CPCG0235Frame$ReadPosition, na.rm=TRUE)
#CPCG0235Frame$StrandBias[is.na(CPCG0235Frame$StrandBias)] = mean(CPCG0235Frame$StrandBias, na.rm=TRUE)

#print (sum(is.na(CPCG0235Frame$ReadPosition)) )
#print (sum(is.na(CPCG0235Frame$StrandBias)) )

In [46]:
%%R
sum(is.na(PCSI0044Frame))  #No NA value now

[1] 0


In [47]:
%%R
write.csv(PCSI0044Frame, file = "PCSI0044_imputed.csv")  #Cleaned CSV without NA

In [48]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 hours 6 minutes 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xqb196 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.57 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [49]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/PCSI0044_imputed.csv"
PCSI0044.hex          = h2o.importFile(path = csvPath, destination_frame = "PCSI0044.hex") 
train                 = as.data.frame(PCSI0044.hex)

str(train)

  |======================================================================| 100%
'data.frame':	118394 obs. of  16 variables:
 $ C1             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  522 362 122 146 185 130 148 120 253 90 ...
 $ NonRefAllele   : int  12 152 54 33 97 22 32 61 97 22 ...
 $ BaseQual       : int  16153 9437 3685 4054 8002 4592 5697 5812 9971 3126 ...
 $ TumourCoverage : int  620 614 185 193 287 157 186 187 357 113 ...
 $ NormalCoverage : int  961 871 219 233 172 118 90 90 210 101 ...
 $ MapQual        : num  17 17 17 17 17 16 16 16 37 0 ...
 $ ReadPosition   : num  22 81 45 46 35 67 57 59 67 59 ...
 $ Trinucleotide  : Factor w/ 33 levels "ACA","ACC","ACG",..: 30 32 25 13 11 10 9 31 18 19 ...
 $ HomopolymerRate: num  62.06 4.73 4.84 4.9 2.75 ...
 $ GCcontent      : num  0.343 0.507 0.502 0.498 0.642 ...
 $ GenomicLocation: Factor w/ 15 levels "downstream","exonic",..: NA NA NA NA 7 NA 7 7 NA NA ...
 $ Distance

In [50]:
%%R

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%


In [51]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [52]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
        variable relative_importance scaled_importance percentage
1   NonRefAllele            1.000000          1.000000   0.224545
2       BaseQual            0.489538          0.489538   0.109923
3 NormalCoverage            0.292588          0.292588   0.065699
4      RefAllele            0.182690          0.182690   0.041022
5 TumourCoverage            0.165102          0.165102   0.037073

---
                     variable relative_importance scaled_importance percentage
56          Trinucleotide.CCT            0.012536          0.012536   0.002815
57   GenomicLocation.intronic            0.010406          0.010406   0.002337
58 GenomicLocation.intergenic            0.007761          0.007761   0.001743
59               ReadPosition            0.007600          0.007600   0.001706
60                 StrandBias            0.003251          0.003251   0.000730
61  Trinucleotide.missing(NA)            0.000000          0.000000   0.000000


In [53]:
%%R
write.table(var_importance, file="PCSI0044_variable_importance.csv")

In [54]:
var_importance = pd.read_csv(open('PCSI0044_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [55]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.224545
2,BaseQual,0.489538,0.489538,0.109923
3,NormalCoverage,0.292588,0.292588,0.065699
4,RefAllele,0.182690,0.182690,0.041022
5,TumourCoverage,0.165102,0.165102,0.037073


In [56]:
GenomicFeatures_PCSI0044[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10038,522,12,16153,620,961,17,22,TTA,62.057971,0.343284,NaN,20885,0.820968
1,chr1,10109,362,152,9437,614,871,17,81,TTG,4.730000,0.507463,NaN,20814,0.749186
2,chr1,10177,122,54,3685,185,219,17,45,GTT,4.838384,0.502488,NaN,20746,0.281081
3,chr1,10180,146,33,4054,193,233,17,46,CTA,4.897959,0.497512,NaN,20743,0.264249
4,chr1,15190,185,97,8002,287,172,17,35,CCG,2.750000,0.641791,ncRNA_intronic,15733,0.463415


In [57]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.48953807353973405,
 'DistanceSNP': 0.075583405792713207,
 'GCcontent': 0.017046337947249399,
 'GenomicLocation.UTR3': 0.018351698294281998,
 'GenomicLocation.UTR5': 0.0756726935505867,
 'GenomicLocation.downstream': 0.037893891334533698,
 'GenomicLocation.exonic': 0.021304534748196598,
 'GenomicLocation.exonic;splicing': 0.064694978296756703,
 'GenomicLocation.intergenic': 0.0077611575834453097,
 'GenomicLocation.intronic': 0.010405756533145899,
 'GenomicLocation.missing(NA)': 0.022662183269858398,
 'GenomicLocation.ncRNA_UTR3': 0.052873715758323697,
 'GenomicLocation.ncRNA_UTR5': 0.031074021011590999,
 'GenomicLocation.ncRNA_exonic': 0.089108295738697094,
 'GenomicLocation.ncRNA_intronic': 0.0139008304104209,
 'GenomicLocation.ncRNA_splicing': 0.040834594517946202,
 'GenomicLocation.splicing': 0.058591742068529101,
 'GenomicLocation.upstream': 0.043934524059295703,
 'GenomicLocation.upstream;downstream': 0.052855800837278401,
 'HomopolymerRate': 0.15240022540092499,
 'M

In [58]:
GenomicFeatures_PCSI0044.columns

Index([u'CHROM', u'POS', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'DistanceSNP', u'StrandBias'], dtype='object')

In [59]:
GenomicFeatures_PCSI0044.dtypes

CHROM               object
POS                  int64
RefAllele            int64
NonRefAllele         int64
BaseQual             int64
TumourCoverage       int64
NormalCoverage       int64
MapQual            float64
ReadPosition       float64
Trinucleotide       object
HomopolymerRate    float64
GCcontent          float64
GenomicLocation     object
DistanceSNP          int64
StrandBias         float64
dtype: object

In [60]:
GenomicFeatures_PCSI0044.iloc[0,13]

20885

In [61]:
GenomicFeatures_PCSI0044[:2]
#print GenomicFeatures_IS1.iloc[0,2]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10038,522,12,16153,620,961,17,22,TTA,62.057971,0.343284,NaN,20885,0.820968
1,chr1,10109,362,152,9437,614,871,17,81,TTG,4.730000,0.507463,NaN,20814,0.749186


In [62]:
noOfSample = GenomicFeatures_PCSI0044.shape[0]

PCSI0044_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_PCSI0044.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_PCSI0044.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_PCSI0044.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_PCSI0044.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_PCSI0044.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_PCSI0044.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_PCSI0044.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_PCSI0044.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_PCSI0044.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_PCSI0044.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_PCSI0044.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_PCSI0044.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_PCSI0044.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_PCSI0044.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_PCSI0044.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_PCSI0044.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' Distance=',Distance
    #print 'StrandBias=',StrandBias
    PCSI0044_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [63]:
#Sorting dictionary
PCSI0044_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in PCSI0044_Dict.items()],reverse=True)]

In [64]:
PCSI0044_Dict_Frame = pd.DataFrame(PCSI0044_Dict1, columns=['POS', 'Value'])

In [65]:
PCSI0044_Dict_Frame.to_csv("PCSI0044_Dict_Frame.csv")

In [95]:
PCSI0044_Dict_Frame = pd.read_csv("PCSI0044_Dict_Frame.csv")

In [96]:
PCSI0044_Dict_Frame[:10]

,Unnamed: 0,POS,Value
0,0,58869650,2339053.791079
1,1,58834107,2319959.442462
2,2,58837886,2319894.153539
3,3,58837910,2319612.264680
4,4,58837875,2318704.941375
5,5,58855159,2314752.940285
6,6,58837856,2314007.250568
7,7,58837892,2313531.945171
8,8,58837854,2312976.072535
9,9,58837530,2308411.309343


In [97]:
GenomicFeatures_PCSI0044_1 = pd.concat([PCSI0044.iloc[:,0:16], PCSI0044_Y_Val_1],axis=1)

In [99]:
rows = PCSI0044_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_PCSI0044_Top20 = GenomicFeatures_PCSI0044_1.loc[GenomicFeatures_PCSI0044_1['POS_x'].isin(rows)]

[58869650 58834107 58837886 58837910 58837875 58855159 58837856 58837892
 58837854 58837530 58915990 58905084 58915982 58865396 58905377 58865376
 58829728 58906845 58912822 58865373]


In [101]:
GenomicFeatures_PCSI0044_Top20[:10]

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,Label
118105,chrY,58829728,1903,44,62182,2012,1717,0,40,GCA,2.427536,0.388060,NaN,30027203,0.559145,58829727,0
118106,chrY,58834107,3093,70,98272,3213,2720,0,70,TCC,2.101351,0.358209,NaN,30031582,0.205415,58834106,0
118109,chrY,58837530,2153,230,74627,2440,2405,0,41,TTC,2.514706,0.412935,NaN,30035005,0.522131,58837529,0
118110,chrY,58837854,2644,23,83750,2776,2870,0,41,ATT,2.434783,0.393035,NaN,30035329,0.584294,58837853,0
118111,chrY,58837856,2666,37,85784,2817,2906,0,40,GCA,2.402878,0.398010,NaN,30035331,0.584665,58837855,0
118112,chrY,58837875,2932,17,95077,3059,3174,0,41,CTA,2.395683,0.393035,NaN,30035350,0.610984,58837874,0
118113,chrY,58837886,3031,36,97309,3189,3301,0,45,GCA,2.395683,0.402985,NaN,30035361,0.624020,58837885,0
118114,chrY,58837892,2972,59,84329,3136,3260,0,49,TCG,2.395683,0.402985,NaN,30035367,0.632653,58837891,0
118115,chrY,58837910,2949,67,96805,3121,3158,0,53,GCA,2.467153,0.417910,NaN,30035385,0.613585,58837909,0
118134,chrY,58855159,2624,164,83993,2831,3565,0,57,ATG,2.740458,0.393035,NaN,30052634,0.148357,58855158,0


In [69]:
GenomicFeatures_PCSI0044_Top20.to_csv('GenomicFeatures_PCSI0044_Top20.csv')